In [187]:
import gadata_funcs,pdb,re
import pandas as pd
import numpy as np
from gadata_funcs import date2days
from importlib import reload
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

%matplotlib notebook

In [21]:
# Loading in data and adding day info
df=pd.read_csv("gadata.csv",dtype={'date':str})
df2=pd.read_csv("/home/wlwoon/Dropbox/chicken_work/impersuasion_stuff/withSalesData/sales_order_line_timestamped.csv")
df['day']=df.date.apply(gadata_date2days)
df2['day']=df2.transaction_date.apply(salesdata_date2days)

In [64]:
# The ranges for the feature generate and target windows respectively
inFeatRange=set(list(range(120,150)))
labelRange=set(list(range(150,170)))

In [192]:
# Preparing to extract features
dff=df[df.day.isin(inFeatRange)]
dff2=df2[df2.day.isin(labelRange)]
dff2=dff2[dff2.customer_id.isin(set(dff.user_id))]
dg=dff.groupby('user_id')

In [194]:
# Feature extraction

# x (inputs)
xdf=dg.hits.count()
xdf=pd.concat((xdf,dg.country.first()),axis=1)
xdf['malaysia']=(xdf.country=='Malaysia').astype(int)
xdf['malsing']=(xdf.country.isin(['Malaysia','Singapore'])).astype(int)

# y (targets)
ydf=pd.DataFrame(xdf.index.isin(set(dff2.customer_id)).astype(int),index=xdf.index)

In [203]:
# Just the relevant features, and formatting for easier use
x=xdf[['hits','malsing']].values
y=ydf.values.reshape((-1,))

In [204]:
rf=RandomForestClassifier(n_estimators=20)

In [207]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.1)

rf.fit(xtrain,ytrain)
rf.score(xtest,ytest)

0.7157968970380818

In [135]:
#dg.hits.count()[dg.hits.nunique()>1]
dg.hits.value_counts()[13486]

hits
1    108
2     36
Name: hits, dtype: int64